# Aggregation pipeline
This Notebook is about getting to know Mongo's accident analysis pipeline. It's presented as a series of exercises that generally replicate queries you've already perfromed on the accidents dataset.

In [1]:
# Import the required libraries

import pymongo
import datetime
import collections

import pandas as pd
import scipy.stats

In [2]:
# Open a connection to the Mongo server, open the accidents database and name the collections of accidents and labels
client = pymongo.MongoClient('mongodb://localhost:27351/')

db = client.accidents
accidents = db.accidents
labels = db.labels

In [3]:
# Load the expanded names of keys and human-readable codes into memory

expanded_name = collections.defaultdict(str)
for e in labels.find({'expanded': {"$exists": True}}):
    expanded_name[e['label']] = e['expanded']
    
label_of = collections.defaultdict(str)
for l in labels.find({'codes': {"$exists": True}}):
    for c in l['codes']:
        try:
            label_of[l['label'], int(c)] = l['codes'][c]
        except ValueError: 
            label_of[l['label'], c] = l['codes'][c]

## An example
First, an example of using the aggregation pipeline to get you started. 

This pipeline finds all the accidents at 30mph or above, groups them by speed limit, and finds the number of accidents at each speed limit.

* `$match` is first and selects the relevant accidents.
* `$group` is next and groups the accidents. The `_id` is the value being grouped on. `{'$sum': 1}` is the value assigned to each member of the group, and how those values are combined. This is the idiom for counting members in a group.

In [4]:
# Pull out all the accidents at 30mph or above, group by speed, and show totals at each speed.
pipeline = [{'$match': {'Speed_limit': {'$gte': 30}}},
            {'$group': {'_id': '$Speed_limit',
                        'num_accidents': {'$sum': 1}}}]
results = list(accidents.aggregate(pipeline))
results

[{'_id': 70, 'num_accidents': 10022},
 {'_id': 40, 'num_accidents': 11914},
 {'_id': 30, 'num_accidents': 94995},
 {'_id': 50, 'num_accidents': 5220},
 {'_id': 60, 'num_accidents': 21172}]

We can now put the results in a _pandas_ DataFrame.

In [5]:
results_df = pd.DataFrame(results)
results_df.set_index('_id', inplace=True)
results_df.index.name = 'speed_limit'
results_df

,num_accidents
speed_limit,
70,10022
40,11914
30,94995
50,5220
60,21172


### Activity note
With all the activities below, build up the pipeline in stages. The `$limit` operator is your friend here: it will allow you to see what the pipeline produces without being overwhelmed by thousands of items.

### Activity 1
Find all the accidents at 30mph or above, group them by speed limit and accident severity, and find the number of accidents at each speed limit/severity combination.

Hint: If you give multiple keys for a single `$group` operation, it will return one group for each combination of those keys.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [10]:
pipeline = [{'$match': {'Speed_limit': {'$gte': 30}}},
            {'$group': {'_id': {'Speed_limit': '$Speed_limit',
                                'Accident_Severity': '$Accident_Severity'},
                       'num_accidents': {'$sum': 1}}}]
results = list(accidents.aggregate(pipeline))
results_df = pd.DataFrame(results)
results_df


,_id,num_accidents
0,"{'Accident_Severity': 1, 'Speed_limit': 70}",198
1,"{'Accident_Severity': 1, 'Speed_limit': 60}",599
2,"{'Accident_Severity': 2, 'Speed_limit': 60}",4157
3,"{'Accident_Severity': 3, 'Speed_limit': 40}",9961
4,"{'Accident_Severity': 3, 'Speed_limit': 60}",16416
5,"{'Accident_Severity': 1, 'Speed_limit': 30}",582
6,"{'Accident_Severity': 2, 'Speed_limit': 30}",12686
7,"{'Accident_Severity': 3, 'Speed_limit': 70}",8667
8,"{'Accident_Severity': 3, 'Speed_limit': 30}",81727
9,"{'Accident_Severity': 3, 'Speed_limit': 50}",4356


### Activity 2
Given the number of accidents at each speed limit/severity combination, create a *pandas* DataFrame to store the results. The DataFrame should have columns of severity (as 'Fatal', 'Serious', 'Slight', in that order) and an index of speed limits (in order).

Hint: Convert the result of Activity 1 into a list of dicts, import it into a DataFrame, then use `set_index` and `unstack` to reshape the data.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [12]:
# convert results into a dict
results_long_df= pd.DataFrame([
    {'Accident_Severity': r['_id']['Accident_Severity'],
    'speed_limit': r['_id']['Speed_limit'],
    'num_accidents': r['num_accidents']}
    for r in results])
results_long_df

,Accident_Severity,num_accidents,speed_limit
0,1,198,70
1,1,599,60
2,2,4157,60
3,3,9961,40
4,3,16416,60
5,1,582,30
6,2,12686,30
7,3,8667,70
8,3,81727,30
9,3,4356,50


In [15]:
# Reshape the dataframe
results_df = results_long_df.pivot(index="speed_limit", columns='Accident_Severity', values='num_accidents')
results_df.columns = [label_of['Accident_Severity', c] for c in results_df.columns]
results_df.columns.name = 'Severity'
results_df.index.name = 'Speed limit'

results_df

Severity,Fatal,Serious,Slight
Speed limit,,,
30,582,12686,81727
40,155,1798,9961
50,94,770,4356
60,599,4157,16416
70,198,1157,8667


### Activity 3
Group the accidents by number of vehicles and number of casualties. Find the number of accidents at each vehicle/casualty combination. Store the results in a DataFrame with the number of vehicles as the columns and number of casualties as the index. 

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

### HELPER FUNCTION


In [17]:
def results_to_table(results, index_name, column_name, results_name,
                    fillna=None,
                    relabel_index=False, relabel_columns=False,
                    index_label=None, column_label=None):
    # Move items in dicts-of-dicts to the top level
    def flatten(d):
        new_d = {}
        for k in d:
            if isinstance(d[k], dict):
                new_d.update(flatten(d[k]))
            else:
                new_d[k] = d[k]
        return new_d
    
    df = pd.DataFrame([flatten(r) for r in results])
    df = df.pivot(index=index_name, columns=column_name, values=results_name)
    
    # optionally fiddle with names and labels to make the DataFrame pretty.
    if not fillna is None:
        df.fillna(fillna, inplace=True)
    if relabel_columns:
        df.columns = [label_of[column_name, c] for c in df.columns]
    if relabel_index:
        df.index = [label_of[index_name, r] for r in df.index]
    if column_label:
        df.columns.name = column_label
    else:
        df.columns.name = column_name
    if index_label:
        df.index.name = index_label
    else:
        df.index.name = index_name
    
    return df

In [21]:
pipeline = [{'$group': {'_id': {'Number_of_Casualties': '$Number_of_Casualties',
                                'Number_of_Vehicles': '$Number_of_Vehicles'},
                       'num_accidents': {'$sum': 1}}}]
results = list(accidents.aggregate(pipeline))

results_to_table(results, 'Number_of_Casualties', 'Number_of_Vehicles', 'num_accidents', fillna=0)

Number_of_Vehicles,1,2,3,4,5,6,7,8,9,10,11,12,13,16,18
Number_of_Casualties,,,,,,,,,,,,,,,
1,39217.0,65680.0,6168.0,1056.0,197.0,76.0,27.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,3781.0,15035.0,3002.0,716.0,128.0,41.0,14.0,4.0,6.0,3.0,0.0,1.0,0.0,1.0,0.0
3,682.0,4186.0,1203.0,319.0,73.0,25.0,12.0,6.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
4,287.0,1440.0,481.0,166.0,48.0,11.0,2.0,3.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0
5,79.0,521.0,169.0,63.0,29.0,10.0,6.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,22.0,183.0,72.0,46.0,13.0,4.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7.0,52.0,28.0,12.0,7.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,5.0,24.0,12.0,9.0,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,2.0,11.0,5.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Activity 4
Group the accidents by severity and junction type (`Junction_Detail`). Find the number of accidents at each junction/severity combination. Store the results in a DataFrame with the severities as the columns and junction types as the index. Columns and index should contain the text labels (e.g. Fatal, Serious; Roundabout, Crossroads), not the codes.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [24]:
pipeline = [{'$group': {'_id': {'Severity': '$Accident_Severity',
                               'Junction_Type': '$Junction_Detail'},
                       'num_accidents': {'$sum': 1}}}]

results = list(accidents.aggregate(pipeline))

results_to_table(results, 'Severity', 'Junction_Type', 'num_accidents', fillna=0)

Junction_Type,0,1,2,3,5,6,7,8,9
Severity,,,,,,,,,
1,995,32,11,370,29,87,16,57,40
2,9209,1212,161,6802,217,1824,269,765,442
3,46083,12012,1532,39540,1902,12599,1759,4681,2925


### Activity 5
Find the average number of vehicles and casualties for accidents at each speed limit. Replace the `_id` of each speed limit group with the plain `Speed_limit`.

Store the results in a DataFrame with the averages as the columns and speed limits as the index. 

Hint: Use `$group` to find the total vehicles and casualties, then use `$project` to find the averages and rename the ID.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

### Activity 6
Find number of casualties for each casualty severity / casualty age band combination. 

Store the results in a DataFrame with the severities as the columns and age bands as the index. The columns and index should contain the text labels (e.g. 21-25, 46-55; Fatal, Slight), not the codes.

Hint: Use `$unwind` to examine each `casualty` sub-document in turn.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

### Activity 7
We can use the aggregation pipeline for complex queries. In this exercise, you'll recreate the summary data for finding Spearman's *r* for passenger and driver age bands. 

Find the number of casualties for each driver age band / passenger age band combination. 

Be careful that you only count casualty/driver combinations where the casualty and driver are in the same vehicle. Make sure you exclude combinations where either the driver age band or passenger age band is unknown.

Store the results in a DataFrame with the driver age band as the columns and passenger age bands as the index. The columns and index should contain the text labels (e.g. 21-25, 46-55), not the codes.

Expand the summary results into a list of individual driver/passenger age bands and find Spearman's *r* for these data.

Hint: You can't directly `$match` two values in the same document, so use `$project`'s `$eq` to create a field that flags if a driver/casualty combination comes from one vehicle then `$match` on that field.

The solution is in the [`15.2solutions`](15.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

## Summary
Aggregation pipelines allow complex data-processing tasks to be built up from sequences of simple operations. It continues the trend of moving the data processing to be closer to the data. We started by bringing the data into the client machine for processing (in the form of a spreadsheet or DataFrame). Simple database queries allowed us to bring just the data of interest to the client, but we still had to do the processing and summarisation here. Aggregation pipelines allow us to move much more of the processing to the database, meaning we can end up with just the summary data alone. As datasets get larger, this becomes more important.

## What next?
If you are working through this Notebook as part of an inline exercise, return to the module materials now.

If you are working through this set of Notebooks as a whole, move on to `15.3 Plotting small accidents and roads`.